In [1]:
from cosmology import *
from ktau import *

In [98]:
#Setup Testing Apparatus

def nothing():
    pass

def nothing2():
    print("nothing.")
    pass

func = nothing

def setfunc(_func):
    global func #refer to the global variable
    func = _func

def checkSingleObjectEqual(obj, exp):
    if type(obj) != type(exp):
        return False
    if hasattr(obj, '__dict__'): #complex object
        return obj.__dict__ == exp.__dict__
    else:
        return obj == exp

def testfunc(exp, *args):
    output = func(*args)
    if type(output) != type(exp):
        print("FAILED. Output not same type as input.")
    elif type(exp) == np.ndarray:
        if np.array_equal(output, exp):
            print("PASSED.")
        else:
            print("FAILED. Numpy array differs from expectation.")
    elif type(exp) == list:
        print("PASSED.")
    else:
        print("FAILED.")

In [93]:
#Verify checkSingleObjectEqual works
#TRUE:
print(checkSingleObjectEqual(1, 1))
print(checkSingleObjectEqual(69.5843, 69.5843))
print(checkSingleObjectEqual("$!@&#!*@#7554372", "$!@&#!*@#7554372"))
print(checkSingleObjectEqual(nothing, nothing))
print("------")

#FALSE:
print(checkSingleObjectEqual(1, 2))
print(checkSingleObjectEqual(5.0, 6.0))
print(checkSingleObjectEqual("56712j351824731824", "56712j351824731824  "))
print(checkSingleObjectEqual(nothing, nothing2))

True
True
True
True
------
False
False
False
True


In [106]:
import unittest

def add(a, b):
    return a + b

class TestNotebook(unittest.TestCase):
    
    def test_add(self):
        self.assertEqual(add(2, 2), 5)
        

unittest.main(argv=[''], verbosity=2, exit=False)

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument '/Users/williamhuang/Library/Jupyter/runtime/kernel-v2-792u10vAUpGSz1r.json'


AssertionError: 

In [74]:
def plus1(x):
    return [i+1 for i in x]

setfunc(plus1)
testfunc([3,4,5], [2,3,4])

AttributeError: 'list' object has no attribute '__dict__'